In [3]:
import re
import pandas as pd
from itertools import product

def results_text_to_clipboard_df(text: str):
    """
    Parse lines like:
      [train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.29588704978190755
    Build a DF with columns: event, 5_1, 5_2, 5_3, 10_1, ... 50_3
    Copy DF to clipboard for easy paste into Sheets.
    """
    # event, lb size, set number, score
    rx = re.compile(
        r"\[train\[(.+?)_lb(\d+)_set(\d+)\]\]\s*macro-F1\s*=\s*([0-9]*\.?[0-9]+)",
        flags=re.IGNORECASE
    )

    rows = []
    for m in rx.finditer(text):
        event, lb, setnum, score = m.group(1), m.group(2), m.group(3), m.group(4)
        rows.append((event, int(lb), int(setnum), float(score)))

    if not rows:
        print("No matches found. Check the text format or regex.")
        return None

    # Build wide table
    df_long = pd.DataFrame(rows, columns=["event", "lb", "set", "score"])
    # Keep only the requested lb/set grid
    wanted_lbs  = [5, 10, 25, 50]
    wanted_sets = [1, 2, 3]
    df_long = df_long[df_long["lb"].isin(wanted_lbs) & df_long["set"].isin(wanted_sets)]

    # Pivot to event rows, columns like "5_1"
    df_wide = df_long.assign(col=lambda d: d["lb"].astype(str) + "_" + d["set"].astype(str)) \
                     .pivot_table(index="event", columns="col", values="score", aggfunc="first")

    # Ensure full cartesian column order exists
    all_cols = [f"{lb}_{s}" for lb, s in product(wanted_lbs, wanted_sets)]
    df_wide = df_wide.reindex(columns=all_cols)

    # Put 'event' as an actual column (since you asked for it in the output)
    df_out = df_wide.reset_index()

    # Copy to clipboard
    # df_out.to_clipboard(index=False)
    print("✅ Copied to clipboard. Preview:")
    print(df_out)
    return df_out

# ---- Example usage ----
text = """[train[hurricane_florence_2018_lb5_set1]] macro-F1 = 0.2887379540393716
[train[hurricane_florence_2018_lb5_set2]] macro-F1 = 0.3796424767528141
[train[hurricane_florence_2018_lb5_set3]] macro-F1 = 0.35779673556331987
[train[kaikoura_earthquake_2016_lb5_set1]] macro-F1 = 0.5312856124769608
[train[kaikoura_earthquake_2016_lb5_set2]] macro-F1 = 0.5834699984273525
[train[kaikoura_earthquake_2016_lb5_set3]] macro-F1 = 0.45804183435740203
[train[hurricane_maria_2017_lb5_set1]] macro-F1 = 0.3465568862122409
[train[hurricane_maria_2017_lb5_set2]] macro-F1 = 0.518389258239454
[train[hurricane_maria_2017_lb5_set3]] macro-F1 = 0.3919864921595617
[train[hurricane_harvey_2017_lb5_set1]] macro-F1 = 0.3231846275106901
[train[hurricane_harvey_2017_lb5_set2]] macro-F1 = 0.298217865522242
[train[hurricane_harvey_2017_lb5_set3]] macro-F1 = 0.4730706351506323
[train[kerala_floods_2018_lb5_set1]] macro-F1 = 0.10556353366885479
[train[kerala_floods_2018_lb5_set2]] macro-F1 = 0.2845219897569338
[train[kerala_floods_2018_lb5_set3]] macro-F1 = 0.3495951777155864
[train[california_wildfires_2018_lb5_set1]] macro-F1 = 0.20337942163330336
[train[california_wildfires_2018_lb5_set2]] macro-F1 = 0.08930943624177473
[train[california_wildfires_2018_lb5_set3]] macro-F1 = 0.21358442895953536
[train[cyclone_idai_2019_lb5_set1]] macro-F1 = 0.21759307528219396
[train[cyclone_idai_2019_lb5_set2]] macro-F1 = 0.27329940970985095
[train[cyclone_idai_2019_lb5_set3]] macro-F1 = 0.3978525275448155
[train[hurricane_dorian_2019_lb5_set1]] macro-F1 = 0.11995460581094816
[train[hurricane_dorian_2019_lb5_set2]] macro-F1 = 0.2755395458030017
[train[hurricane_dorian_2019_lb5_set3]] macro-F1 = 0.3385793300458502
[train[hurricane_irma_2017_lb5_set1]] macro-F1 = 0.2528436082018834
[train[hurricane_irma_2017_lb5_set2]] macro-F1 = 0.26807342684451735
[train[hurricane_irma_2017_lb5_set3]] macro-F1 = 0.03682606352712736
[train[canada_wildfires_2016_lb5_set1]] macro-F1 = 0.45194745616976695
[train[canada_wildfires_2016_lb5_set2]] macro-F1 = 0.3616085805300269
[train[canada_wildfires_2016_lb5_set3]] macro-F1 = 0.5023152146540905
[train[hurricane_florence_2018_lb10_set1]] macro-F1 = 0.45263977965033203
[train[hurricane_florence_2018_lb10_set2]] macro-F1 = 0.5642160280325692
[train[hurricane_florence_2018_lb10_set3]] macro-F1 = 0.4976467929223817
[train[kaikoura_earthquake_2016_lb10_set1]] macro-F1 = 0.6967617459778296
[train[kaikoura_earthquake_2016_lb10_set2]] macro-F1 = 0.5751998314936443
[train[kaikoura_earthquake_2016_lb10_set3]] macro-F1 = 0.5896565989046363
[train[hurricane_maria_2017_lb10_set1]] macro-F1 = 0.42809874748343285
[train[hurricane_maria_2017_lb10_set2]] macro-F1 = 0.5165795079922668
[train[hurricane_maria_2017_lb10_set3]] macro-F1 = 0.5404767647253562
[train[hurricane_harvey_2017_lb10_set1]] macro-F1 = 0.5506553762017309
[train[hurricane_harvey_2017_lb10_set2]] macro-F1 = 0.4265437241084266
[train[hurricane_harvey_2017_lb10_set3]] macro-F1 = 0.51343808500438
[train[kerala_floods_2018_lb10_set1]] macro-F1 = 0.3420190101439889
[train[kerala_floods_2018_lb10_set2]] macro-F1 = 0.37472502707833566
[train[kerala_floods_2018_lb10_set3]] macro-F1 = 0.28365004594076765
[train[california_wildfires_2018_lb10_set1]] macro-F1 = 0.5460094166607986
[train[california_wildfires_2018_lb10_set2]] macro-F1 = 0.5244834196833502
[train[california_wildfires_2018_lb10_set3]] macro-F1 = 0.48505057398263884
[train[cyclone_idai_2019_lb10_set1]] macro-F1 = 0.507865205124454
[train[cyclone_idai_2019_lb10_set2]] macro-F1 = 0.4855657380504871
[train[cyclone_idai_2019_lb10_set3]] macro-F1 = 0.46419031828711865
[train[hurricane_dorian_2019_lb10_set1]] macro-F1 = 0.48141492893098775
[train[hurricane_dorian_2019_lb10_set2]] macro-F1 = 0.33784575638921543
[train[hurricane_dorian_2019_lb10_set3]] macro-F1 = 0.48929367467409335
[train[hurricane_irma_2017_lb10_set1]] macro-F1 = 0.394502597808435
[train[hurricane_irma_2017_lb10_set2]] macro-F1 = 0.4778577834466756
[train[hurricane_irma_2017_lb10_set3]] macro-F1 = 0.5177709121545588
[train[canada_wildfires_2016_lb10_set1]] macro-F1 = 0.5060363974713051
[train[canada_wildfires_2016_lb10_set2]] macro-F1 = 0.5516494918862507
[train[canada_wildfires_2016_lb10_set3]] macro-F1 = 0.5055001026364266
[train[hurricane_florence_2018_lb25_set1]] macro-F1 = 0.5905460419728439
[train[hurricane_florence_2018_lb25_set2]] macro-F1 = 0.6495476844948299
[train[hurricane_florence_2018_lb25_set3]] macro-F1 = 0.5797061760179615
[train[kaikoura_earthquake_2016_lb25_set1]] macro-F1 = 0.7048242187332115
[train[kaikoura_earthquake_2016_lb25_set2]] macro-F1 = 0.6521427891911844
[train[kaikoura_earthquake_2016_lb25_set3]] macro-F1 = 0.7245036796335711
[train[hurricane_maria_2017_lb25_set1]] macro-F1 = 0.5232798851623177
[train[hurricane_maria_2017_lb25_set2]] macro-F1 = 0.6464938581568209
[train[hurricane_maria_2017_lb25_set3]] macro-F1 = 0.6593532270597763
[train[hurricane_harvey_2017_lb25_set1]] macro-F1 = 0.6442100533174232
[train[hurricane_harvey_2017_lb25_set2]] macro-F1 = 0.6504636569305908
[train[hurricane_harvey_2017_lb25_set3]] macro-F1 = 0.6046562812662017
[train[kerala_floods_2018_lb25_set1]] macro-F1 = 0.5165070250495727
[train[kerala_floods_2018_lb25_set2]] macro-F1 = 0.48908471468903136
[train[kerala_floods_2018_lb25_set3]] macro-F1 = 0.5391429156866219
[train[california_wildfires_2018_lb25_set1]] macro-F1 = 0.6049091434480889
[train[california_wildfires_2018_lb25_set2]] macro-F1 = 0.686293386344448
[train[california_wildfires_2018_lb25_set3]] macro-F1 = 0.501630646677903
[train[cyclone_idai_2019_lb25_set1]] macro-F1 = 0.5635778223278587
[train[cyclone_idai_2019_lb25_set2]] macro-F1 = 0.5715306354554831
[train[cyclone_idai_2019_lb25_set3]] macro-F1 = 0.5267364855399326
[train[hurricane_dorian_2019_lb25_set1]] macro-F1 = 0.5285167078839417
[train[hurricane_dorian_2019_lb25_set2]] macro-F1 = 0.5668766004360627
[train[hurricane_dorian_2019_lb25_set3]] macro-F1 = 0.569162292532348
[train[hurricane_irma_2017_lb25_set1]] macro-F1 = 0.5847476440492532
[train[hurricane_irma_2017_lb25_set2]] macro-F1 = 0.5858808129771388
[train[hurricane_irma_2017_lb25_set3]] macro-F1 = 0.5642270159157039
[train[canada_wildfires_2016_lb25_set1]] macro-F1 = 0.5674894432134732
[train[canada_wildfires_2016_lb25_set2]] macro-F1 = 0.5671260672825289
[train[canada_wildfires_2016_lb25_set3]] macro-F1 = 0.5820423216451145
[train[hurricane_florence_2018_lb50_set1]] macro-F1 = 0.6703828209996467
[train[hurricane_florence_2018_lb50_set2]] macro-F1 = 0.6964753587230147
[train[hurricane_florence_2018_lb50_set3]] macro-F1 = 0.6727220988113758
[train[kaikoura_earthquake_2016_lb50_set1]] macro-F1 = 0.7115119703051755
[train[kaikoura_earthquake_2016_lb50_set2]] macro-F1 = 0.5965007608821341
[train[kaikoura_earthquake_2016_lb50_set3]] macro-F1 = 0.7198347897562348
[train[hurricane_maria_2017_lb50_set1]] macro-F1 = 0.6644414734084503
[train[hurricane_maria_2017_lb50_set2]] macro-F1 = 0.6528383737781351
[train[hurricane_maria_2017_lb50_set3]] macro-F1 = 0.6541884632818685
[train[hurricane_harvey_2017_lb50_set1]] macro-F1 = 0.6346137658946402
[train[hurricane_harvey_2017_lb50_set2]] macro-F1 = 0.6661884770941147
[train[hurricane_harvey_2017_lb50_set3]] macro-F1 = 0.7010312131475435
[train[kerala_floods_2018_lb50_set1]] macro-F1 = 0.4843473400603119
[train[kerala_floods_2018_lb50_set2]] macro-F1 = 0.5226633138658959
[train[kerala_floods_2018_lb50_set3]] macro-F1 = 0.551314715227139
[train[california_wildfires_2018_lb50_set1]] macro-F1 = 0.6451577050086877
[train[california_wildfires_2018_lb50_set2]] macro-F1 = 0.6323067768983727
[train[california_wildfires_2018_lb50_set3]] macro-F1 = 0.5948300448278726
[train[cyclone_idai_2019_lb50_set1]] macro-F1 = 0.584553174853912
[train[cyclone_idai_2019_lb50_set2]] macro-F1 = 0.5506746652865333
[train[cyclone_idai_2019_lb50_set3]] macro-F1 = 0.5631697930522161
[train[hurricane_dorian_2019_lb50_set1]] macro-F1 = 0.567534379889629
[train[hurricane_dorian_2019_lb50_set2]] macro-F1 = 0.586013106036048
[train[hurricane_dorian_2019_lb50_set3]] macro-F1 = 0.565094779107305
[train[hurricane_irma_2017_lb50_set1]] macro-F1 = 0.6343815130823055
[train[hurricane_irma_2017_lb50_set2]] macro-F1 = 0.6305784214875539
[train[hurricane_irma_2017_lb50_set3]] macro-F1 = 0.6448860432372773
[train[canada_wildfires_2016_lb50_set1]] macro-F1 = 0.5513272764415593
[train[canada_wildfires_2016_lb50_set2]] macro-F1 = 0.5192685957264989
[train[canada_wildfires_2016_lb50_set3]] macro-F1 = 0.49744513533377943"""
results_text_to_clipboard_df(text)


✅ Copied to clipboard. Preview:
col                      event       5_1       5_2       5_3      10_1  \
0    california_wildfires_2018  0.203379  0.089309  0.213584  0.546009   
1        canada_wildfires_2016  0.451947  0.361609  0.502315  0.506036   
2            cyclone_idai_2019  0.217593  0.273299  0.397853  0.507865   
3        hurricane_dorian_2019  0.119955  0.275540  0.338579  0.481415   
4      hurricane_florence_2018  0.288738  0.379642  0.357797  0.452640   
5        hurricane_harvey_2017  0.323185  0.298218  0.473071  0.550655   
6          hurricane_irma_2017  0.252844  0.268073  0.036826  0.394503   
7         hurricane_maria_2017  0.346557  0.518389  0.391986  0.428099   
8     kaikoura_earthquake_2016  0.531286  0.583470  0.458042  0.696762   
9           kerala_floods_2018  0.105564  0.284522  0.349595  0.342019   

col      10_2      10_3      25_1      25_2      25_3      50_1      50_2  \
0    0.524483  0.485051  0.604909  0.686293  0.501631  0.645158  0.632307   

col,event,5_1,5_2,5_3,10_1,10_2,10_3,25_1,25_2,25_3,50_1,50_2,50_3
0,california_wildfires_2018,0.203379,0.089309,0.213584,0.546009,0.524483,0.485051,0.604909,0.686293,0.501631,0.645158,0.632307,0.594830
1,canada_wildfires_2016,0.451947,0.361609,0.502315,0.506036,0.551649,0.505500,0.567489,0.567126,0.582042,0.551327,0.519269,0.497445
2,cyclone_idai_2019,0.217593,0.273299,0.397853,0.507865,0.485566,0.464190,0.563578,0.571531,0.526736,0.584553,0.550675,0.563170
3,hurricane_dorian_2019,0.119955,0.275540,0.338579,0.481415,0.337846,0.489294,0.528517,0.566877,0.569162,0.567534,0.586013,0.565095
4,hurricane_florence_2018,0.288738,0.379642,0.357797,0.452640,0.564216,0.497647,0.590546,0.649548,0.579706,0.670383,0.696475,0.672722
5,hurricane_harvey_2017,0.323185,0.298218,0.473071,0.550655,0.426544,0.513438,0.644210,0.650464,0.604656,0.634614,0.666188,0.701031
6,hurricane_irma_2017,0.252844,0.268073,0.036826,0.394503,0.477858,0.517771,0.584748,0.585881,0.564227,0.634382,0.630578,0.644886
7,hurricane_maria_2017,0.346557,0.518389,0.391986,0.428099,0.516580,0.540477,0.523280,0.646494,0.659353,0.664441,0.652838,0.654188
8,kaikoura_earthquake_2016,0.531286,0.583470,0.458042,0.696762,0.575200,0.589657,0.704824,0.652143,0.724504,0.711512,0.596501,0.719835
9,kerala_floods_2018,0.105564,0.284522,0.349595,0.342019,0.374725,0.283650,0.516507,0.489085,0.539143,0.484347,0.522663,0.551315
